# Imports

In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
pd.set_option('float_format', '{:f}'.format)
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', 99)
import pandas_bokeh
pandas_bokeh.output_file("data/bokeh_output.html")
from qgrid import show_grid
# import finplot as fplt

import polygon_rest_api as papi
import polygon_ws_api as pws
import polygon_df as pdf
import polygon_ds as pds
import polygon_s3 as ps3
import polygon_backfill_flow as fb

import bar_samples as bs
import bar_labels as bl
import bar_ray_workflow as bm

import utils_filters as ft

import matplotlib as mpl
from matplotlib import style
style.use('fivethirtyeight')
mpl.rcParams['figure.figsize'] = [20, 7]

# import ipdb
# ipdb.set_trace(context=10)

# df.style.background_gradient(cmap='coolwarm', axis=None, vmin=-1, vmax=1)  # 'Blues', 'coolwarm', 'RdBu_r' & 'BrBG' are other good diverging colormaps
# pd.set_option('precision', 2)

# .plot_bokeh(sizing_mode="scale_height")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Backfill Data

In [26]:
clust_gold = ['AEM', 'AGI', 'ASA', 'AU', 'AUY', 'BTG', 'DRD', 'EGO', 'FNV', 'GFI', 'HMY', 'IAG', 'KGC', 'KL', 'NEM', 'NG', 'OR', 'PAAS', 'RGLD', 'SA', 'SAND', 'SSRM', 'WPM']
clust_silver = ['AG', 'AXU', 'BVN', 'CDE', 'EXK', 'FSM', 'GORO', 'GSS', 'HL', 'MAG', 'MUX', 'PVG', 'SVM', 'USAS']
all_sym = clust_gold + clust_silver + ['VTI', 'GLD']

In [41]:
datetime.today()

datetime.datetime(2020, 12, 24, 21, 44, 25, 266839)

In [37]:
from datetime import datetime, timedelta, date
(date.today() - timedelta(days=1)).isoformat()

'2020-12-23'

In [33]:
pd.Series(ps3.list_symbol_dates('market','daily'))

0       2016-01-04
1       2016-01-05
2       2016-01-06
3       2016-01-07
4       2016-01-08
           ...    
1249    2020-12-17
1250    2020-12-18
1251    2020-12-21
1252    2020-12-22
1253    2020-12-23
Length: 1254, dtype: object

In [31]:
# backfill market daily

fb.run_flow(
    symbols=['market'],
    tick_type='daily',
    start_date='2016-01-01',
    n_workers=4,
    threads_per_worker=8
)

[2020-12-24 12:04:36-0800] INFO - prefect.FlowRunner | Beginning Flow run for 'backfill-flow'
[2020-12-24 12:04:36-0800] INFO - prefect.TaskRunner | Task 'start_date': Starting task run...
[2020-12-24 12:04:36-0800] INFO - prefect.TaskRunner | Task 'start_date': Finished task run for task with final state: 'Success'
[2020-12-24 12:04:36-0800] INFO - prefect.TaskRunner | Task 'symbols': Starting task run...
[2020-12-24 12:04:36-0800] INFO - prefect.TaskRunner | Task 'end_date': Starting task run...
[2020-12-24 12:04:36-0800] INFO - prefect.TaskRunner | Task 'symbols': Finished task run for task with final state: 'Success'
[2020-12-24 12:04:36-0800] INFO - prefect.TaskRunner | Task 'tick_type': Starting task run...
[2020-12-24 12:04:36-0800] INFO - prefect.TaskRunner | Task 'end_date': Finished task run for task with final state: 'Success'
[2020-12-24 12:04:36-0800] INFO - prefect.TaskRunner | Task 'tick_type': Finished task run for task with final state: 'Success'
[2020-12-24 12:04:37-0

<Success: "All reference tasks succeeded.">

In [ ]:
# backfill trades

fb.run_flow(
    symbols=all_sym,
    tick_type='trades', 
    start_date='2019-01-01', 
    n_workers=2, 
    threads_per_worker=4
)

# Sample Ticks

In [ ]:
symbol = 'VTI'
start_date = '2020-11-01'
end_date = '2020-11-31'
date = '2020-11-13'

ticks_df = ps3.fetch_date_df(symbol, date, tick_type='trades')
daily_df = bm.get_symbol_vol_filter(symbol, start_date, end_date)

In [ ]:
rs = daily_df[daily_df.date == date]['range_jma_lag'].values[0] / 30

thresh = {
    # max thresholds
    'renko_size': rs,
    'renko_return': 'jma_return',
    'renko_reveral_multiple': 2,
    'max_duration_sec': 60*60*2,  # 2hours
    
    # min thresholds
    'min_duration_sec': 17,
    'min_tick_count': 30,
}

# sample bars
bars, ticks = bs.build_bars(ticks_df, thresh)

ticks2_df = pd.DataFrame(ticks)
bars_df = pd.DataFrame(bars)

ticks2_df.status.value_counts()

In [ ]:
bars_df.duration_td.describe(percentiles=[.01,.1,.8,.95])

In [ ]:
pd.plotting.autocorrelation_plot(bars_df.jma_return)

In [ ]:
# bars_df[['jma_close','jma_high','jma_low']].plot()
bars_df[['jma_wq10', 'jma_wq90','jma_wmean']].plot_bokeh(sizing_mode="scale_height")
# plot_bokeh(sizing_mode="scale_height")

In [ ]:
ticks2_df[ticks2_df.status=='clean'].set_index('date_time').loc[:, ['price', 'jma']].plot()

In [ ]:
ixs += 1000
# ixs = 0
ixe = ixs+1000

ticks2_df[ticks2_df.status=='clean'].loc[ixs:ixe, ['price', 'jma']].plot()

# ticks2_df[['price', 'jma']][ixs:ixe].plot()

In [ ]:
st = '16:20:00'
et = '16:30:00'

dfa = ticks2_df.set_index('date_time').between_time(st, et)[['price', 'jma']]
dfc = ticks2_df[ticks2_df.status=='clean'].set_index('date_time').between_time(st, et)[['price', 'jma']]

dfc[['price', 'jma']].plot()

# post-hoc JMA
# ft.jma_filter_df(dfc, col='price', length=7, power=2)[['price', 'jma']].plot()
# .plot_bokeh(sizing_mode="scale_height")

# Build Bars Workflow

In [ ]:
ray.init(ignore_reinit_error=True, num_cpus=4, dashboard_port=8080)

In [ ]:
daily_bar_stats_df, labeled_bar_dates, stacked_bars_df = bm.bars_workflow_ray(
    symbol, 
    start_date,
    end_date, 
    thresh,
)

In [ ]:
ray.shutdown()